In [112]:
import ctypes
from pathlib import Path
import pandas as pd

In [113]:

class MapsForgeHeader(ctypes.BigEndianStructure):
    _pack_ = 1
    _fields_ = [
        ("magic_byte", ctypes.c_char * 20),
        ("header_size", ctypes.c_uint32),
        ("file_version", ctypes.c_uint32),
        ("file_size", ctypes.c_uint64),
        ("date_of_creation", ctypes.c_uint64),
        ("minLat", ctypes.c_int32),
        ("minLon", ctypes.c_int32),
        ("maxLat", ctypes.c_int32),  # most nothern point
        ("maxLon", ctypes.c_int32),
        ("tile_size", ctypes.c_uint16),
    ]


class ParsedFileName:
    country: str
    state: str
    creationDate: str
    minLong: int
    minLat: int
    sizeNorthSouth: int
    sizeWestEast: int

    def __init__(self, filename):
        self.country = filename[:2]
        self.state = filename[2:6]
        self.creationDate = filename[6:12]
        self.field1 = int(filename[12:15], 36)
        self.field2 = int(filename[15:18], 36)
        self.z1 = int(filename[18:21], 36)
        self.z2 = int(filename[21:24], 36)


def parseHeader(file: Path) -> MapsForgeHeader:
    header = MapsForgeHeader.from_buffer_copy(file.read_bytes())
    return header


def parseFilename(filename) -> ParsedFileName:
    return ParsedFileName(filename)


In [114]:
files = [f for f in Path("./allmaps").glob(("*.map")) if f.is_file()]
data = []


print("--- files ----")
for i in files:
    print(i.name)


--- files ----
asia_China_Anhui-CN010025040256C2ID039045.map
asia_China_Beijing-CN020025040255B2D902R04Q.map
asia_China_Chongqing-CN030025040250B2K903N02Y.map
asia_China_Fujian-CN040025040256Y2MZ03503H.map
asia_China_Gansu-CN05002504024RV2BA0AV08Q.map
asia_China_Guangdong-CN06002504025332OT04U041.map
asia_China_Guangxi-CN07002504024ZO2OF05203U.map
asia_China_Guizhou-CN08002504024Z82MD03V03F.map
asia_China_Hainan-CN090025040252F2SP02M02Z.map
asia_China_Hebei-CN100025040255B2BT049066.map
asia_China_Heilongjiang-CN11002504025702140C60AL.map
asia_China_Henan-CN12002504025352EJ04J06C.map
asia_China_Hongkong-CN130025040255Q2R400E00A.map
asia_China_Hubei-CN140025040251C2JD06F039.map
asia_China_Hunan-CN150025040252I2LS03I03Z.map
asia_China_Inner_Mongolia-CN16002504024V421C0J60EY.map
asia_China_Jiangsu-CN170025040257A2HF03L03X.map
asia_China_Jiangxi-CN180025040255E2LF03J04H.map
asia_China_Jilin-CN19002504025AM28F06J055.map
asia_China_Liaoning-CN200025040258Q2B604L042.map
asia_China_Macao-CN2100

In [115]:
for f in files:
    header = parseHeader(f)
    # fix modified name (my maps are named like Vorarlberg-AT08002303103BS27H00H00P.map)
    humanName, filename = f.name.split("-")
    pf = parseFilename(filename)
    data.append({"file": f, "header": header, "parsedFilename": pf, "Name": humanName})

In [116]:
print("--- bounding boxes from mapsforge files ---")

for entry in data:
    print(
        entry["Name"].ljust(15),
        "minLat",
        str(entry["header"].minLat).ljust(11),
        "minLon",
        str(entry["header"].minLon).ljust(9),
        "maxLat",
        str(entry["header"].maxLat).ljust(11),
        "maxLon",
        entry["header"].maxLon,
    )


--- bounding boxes from mapsforge files ---
asia_China_Anhui minLat 29141565    minLon 114795271 maxLat 34689614    maxLon 119934019
asia_China_Beijing minLat 35177401    minLon 113169069 maxLat 41068783    maxLon 117526822
asia_China_Chongqing minLat 28150997    minLon 105275303 maxLat 32186718    maxLon 111013218
asia_China_Fujian minLat 23531542    minLon 115760366 maxLat 28473141    maxLon 120723117
asia_China_Gansu minLat 32514122    minLon 91926489  maxLat 43386098    maxLon 109115823
asia_China_Guangdong minLat 20040942    minLon 109664659 maxLat 25901797    maxLon 117313692
asia_China_Guangxi minLat 20902293    minLon 104252446 maxLat 26438080    maxLon 112264999
asia_China_Guizhou minLat 24498948    minLon 103571295 maxLat 29321991    maxLon 109681663
asia_China_Hainan minLat 15779891    minLon 108612535 maxLat 20230034    maxLon 112740351
asia_China_Hebei minLat 35177401    minLon 113169069 maxLat 42762641    maxLon 119920098
asia_China_Heilongjiang minLat 42905118    minLon 

In [ ]:
print("--- sorted by maxLat, using field 2 from filename ---")
data = sorted(data, key=lambda entry: entry["header"].maxLat, reverse=True)
prevmaxLatHeader = 0
prevmaxLatFilename = 0
for entry in data:
    maxLatHeader = entry["header"].maxLat
    maxLatFilename = entry["parsedFilename"].field2

    diffHeader =  maxLatHeader - prevmaxLatHeader
    diffFileName = maxLatFilename - prevmaxLatFilename

    print(
        entry["Name"].ljust(13),
        "from header:",
        str(maxLatHeader).ljust(9),
        "diff",
        str(diffHeader).ljust(9) if maxLatHeader != diffHeader else " --- ".ljust(9), 
        "filename:",
        str(maxLatFilename).ljust(9),
        "diff",
        str(diffFileName).ljust(9) if maxLatFilename != diffFileName else " --- ".ljust(9), 
        "diff factor",
        str(diffHeader/diffFileName) if maxLatHeader != diffHeader and diffFileName != 0 else " --- ".ljust(9)
    )
    prevmaxLatHeader = maxLatHeader
    prevmaxLatFilename = maxLatFilename



--- sorted by maxLat, using field 2 from filename ---
europe_Russia_Arkhangelsk_Region from header: 82100000  diff  ---      filename: 611       diff  ---      diff factor  ---     
europe_Russia_Krasnoyarsk_Krai from header: 81510000  diff -590000   filename: 705       diff 94        diff factor -6276.595744680851
europe_Russia_Republic_of_Sakha from header: 77350000  diff -4160000  filename: 1228      diff 523       diff factor -7954.110898661568
europe_Russia_Yamal_Nenets_Autonomous_Region from header: 73760000  diff -3590000  filename: 1557      diff 329       diff factor -10911.854103343465
europe_Norway_Finnmark from header: 71420000  diff -2340000  filename: 1735      diff 178       diff factor -13146.067415730337
europe_Russia_Nenets_Autonomous_Region from header: 70770000  diff -650000   filename: 1781      diff 46        diff factor -14130.434782608696
europe_Norway_Troms_County from header: 70725000  diff -45000    filename: 1784      diff 3         diff factor -15000.0
euro

ZeroDivisionError: division by zero

In [ ]:
print("--- sorted by minLon---")
data = sorted(data, key=lambda entry: entry["header"].minLon, reverse=True)
prevmaxLonHeader = 0
prevmaxLonFilename = 0
for entry in data:
    maxLonHeader = entry["header"].minLon
    maxLonFilename = entry["parsedFilename"].field1

    diffHeader =  maxLonHeader - prevmaxLonHeader
    diffFileName = maxLonFilename - prevmaxLonFilename

    print(
        entry["Name"].ljust(13),
        "from header:",
        str(maxLonHeader).ljust(9),
        "diff",
        str(diffHeader).ljust(9) if maxLonHeader != diffHeader else " --- ".ljust(9), 
        "filename:",
        str(maxLonFilename).ljust(9),
        "diff",
        str(diffFileName).ljust(9) if maxLonFilename != diffFileName else " --- ".ljust(9), 
        "diff factor",
        str(diffHeader/diffFileName) if maxLonHeader != diffHeader and diffFileName != 0 else " --- ".ljust(9)
    )
    prevmaxLonHeader = maxLonHeader
    prevmaxLonFilename = maxLonFilename


--- sorted by minLon---
asia_China_Jilin from header: 121581034 diff  ---      filename: 6862      diff  ---      diff factor  ---     
asia_China_Shanghai from header: 120790317 diff -790717   filename: 6844      diff -18       diff factor 43928.72222222222
asia_China_Liaoning from header: 118590062 diff -2200255  filename: 6794      diff -50       diff factor 44005.1
asia_China_Jiangsu from header: 116287471 diff -2302591  filename: 6742      diff -52       diff factor 44280.596153846156
asia_China_Tianjin from header: 116261961 diff -25510    filename: 6741      diff -1        diff factor 25510.0
asia_China_Heilongjiang from header: 115846659 diff -415302   filename: 6732      diff -9        diff factor 46144.666666666664
asia_China_Fujian from header: 115760366 diff -86293    filename: 6730      diff -2        diff factor 43146.5
asia_China_Anhui from header: 114795271 diff -965095   filename: 6708      diff -22       diff factor 43867.954545454544
asia_China_Shandong from header: 

Seems like Longitute is 0-8192 where greenwhich is 4096 (or bit 13 is the signed bit). Every 44*10^-3 degrees the value increases by 1 in the filename